## YOLOv11 + MobileSAM Pipeline (Box Prompting)
Runs YOLO11 for fire detection and uses MobileSAM with bounding box prompts for segmentation.  
Saves:
- Detection visualizations with confidence scores  
- Segmentation masks  
- Overlays showing detected fire regions

In [ ]:
import os, glob, cv2, torch
import numpy as np
from PIL import Image
from ultralytics import YOLO

from mobile_sam import sam_model_registry, SamPredictor

FRAME_DIR    = ""   # Directory containing input frames
YOLO_MODEL   = ""   # Path to YOLO model weights
SAM_CKPT     = ""   # Path to mobile_sam.pt weights
OUT_DIR      = ""   # Output directory for results
IMG_SIZE     = 960
CONF_THRESH  = 0.3

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

yolo_model = YOLO(YOLO_MODEL).to(DEVICE)

sam = sam_model_registry["vit_t"](checkpoint=SAM_CKPT)
sam.to(device=DEVICE)
sam.eval()
predictor = SamPredictor(sam)

mask_dir = os.path.join(OUT_DIR, "masks")
overlay_dir = os.path.join(OUT_DIR, "overlays")
det_dir = os.path.join(OUT_DIR, "detected_fires")
os.makedirs(mask_dir, exist_ok=True)
os.makedirs(overlay_dir, exist_ok=True)
os.makedirs(det_dir, exist_ok=True)

frame_paths = sorted(
    sum([glob.glob(os.path.join(FRAME_DIR, ext)) for ext in ("*.jpg", "*.png")], [])
)

for idx, path in enumerate(frame_paths):
    img_pil = Image.open(path).convert("RGB")
    img_np  = np.array(img_pil)

    results = yolo_model.predict([img_pil], imgsz=IMG_SIZE, conf=CONF_THRESH)
    boxes_xyxy = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()

    det_vis = img_np.copy()
    for box, conf in zip(boxes_xyxy, confs):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(det_vis, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(det_vis, f"{conf:.2f}", (x1 - 50, y1 + 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.imwrite(os.path.join(det_dir, f"{idx:05d}.png"),
                cv2.cvtColor(det_vis, cv2.COLOR_RGB2BGR))

    predictor.set_image(img_np)
    final_mask = np.zeros(img_np.shape[:2], dtype=np.uint8)

    for box in boxes_xyxy:
        masks, scores, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=box[None, :], 
            multimask_output=False
        )
        mask = masks[0].astype(np.uint8)
        final_mask = np.maximum(final_mask, mask)

    mask_filename = f"{idx:05d}.png"
    cv2.imwrite(os.path.join(mask_dir, mask_filename), final_mask * 255)

    overlay = img_np.copy()
    red = np.zeros_like(img_np); red[:] = (255, 0, 0)
    overlay = np.where(final_mask[..., None] == 1,
                       cv2.addWeighted(overlay, 0.6, red, 0.4, 0),
                       overlay)
    cv2.imwrite(os.path.join(overlay_dir, mask_filename),
                cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

    print(f"[{idx+1}/{len(frame_paths)}] Processed: {os.path.basename(path)}")

print("All frames processed with MobileSAM + YOLO boxes.")


In [ ]:
import os
import glob
import numpy as np
import cv2

GT_MASK_DIR   = ""  # Ground truth masks
PRED_MASK_DIR = ""  # SAM2-predicted masks

def compute_metrics(gt, pred):
    gt = (gt > 127).astype(np.uint8)
    pred = (pred > 127).astype(np.uint8)

    intersection = np.logical_and(gt, pred).sum()
    union = np.logical_or(gt, pred).sum()
    iou = intersection / union if union > 0 else 1.0

    dice = (2 * intersection) / (gt.sum() + pred.sum() + 1e-6)

    mae = np.abs(gt - pred).mean()

    correct = (gt == pred).sum()
    pixel_acc = correct / gt.size

    bg_correct = ((gt == 0) & (pred == 0)).sum()
    bg_total = (gt == 0).sum()
    bg_acc = bg_correct / bg_total if bg_total > 0 else 1.0

    fg_correct = ((gt == 1) & (pred == 1)).sum()
    fg_total = (gt == 1).sum()
    fg_acc = fg_correct / fg_total if fg_total > 0 else 1.0

    mean_acc = (bg_acc + fg_acc) / 2.0

    total_pixels = gt.size
    fw_iou = ((bg_total/total_pixels) * (bg_correct/(bg_total + (pred==1).sum() - fg_correct)) +
              (fg_total/total_pixels) * (fg_correct/(fg_total + (pred==0).sum() - bg_correct)))

    return iou, dice, mae, pixel_acc, mean_acc, fw_iou

gt_paths = sorted(glob.glob(os.path.join(GT_MASK_DIR, "*.png")))
pred_paths = sorted(glob.glob(os.path.join(PRED_MASK_DIR, "*.png")))

ious, dices, maes, accs, mean_accs, fwious = [], [], [], [], [], []

for gt_path, pred_path in zip(gt_paths, pred_paths):
    gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
    pred = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)

    if gt.shape != pred.shape:
        pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_NEAREST)

    iou, dice, mae, acc, mean_acc, fw_iou = compute_metrics(gt, pred)
    ious.append(iou); dices.append(dice); maes.append(mae)
    accs.append(acc); mean_accs.append(mean_acc); fwious.append(fw_iou)

print(f"Mean IoU   : {np.mean(ious):.4f}")
print(f"Mean Dice  : {np.mean(dices):.4f}")
print(f"Mean MAE   : {np.mean(maes):.4f}")
print(f"Pixel Acc  : {np.mean(accs):.4f}")
print(f"Mean Acc   : {np.mean(mean_accs):.4f}")
print(f"FWIoU      : {np.mean(fwious):.4f}")
